In [ ]:
## Goal: 1) extract the <token, id, canonical, variants> triple from the ado.owl
##       2) examine the concepts that are in the ado but not in the nio (this shouldn't be the case but it happens)
##       3) merge the triples with the nio dictionary 

# the ado.owl structure: it has three different ontologies 

# # type 1: AlzheimerOntology
#     <!-- http://scai.fraunhofer.de/AlzheimerOntology#amyloid_precursor_protein -->

#     <owl:Class rdf:about="&AlzheimerOntology;amyloid_precursor_protein">
#         <rdfs:label>amyloid precursor protein</rdfs:label>
#         <rdfs:subClassOf rdf:resource="&AlzheimerOntology;protein"/>
#         <NDDUO:Synonym>amyloid-beta protein precursor</NDDUO:Synonym>
#         <NDDUO:Synonym>APP</NDDUO:Synonym>
#         <NDDUO:Synonym>amyloid precursor-protein</NDDUO:Synonym>
#     </owl:Class>

## turn it into the format below: {token_id:[canonical]}, {token_id:[variants (don't include canonical here)]}
# 	<token id="AlzheimerOntology:Prefrontal_cortex" canonical="prefrontal cortex">
# 		<variant base="prefrontal cortices"/>
# 		<variant base="prefrontal cortex"/>
# 	</token>

## type 2: NDDUO
#     <!-- http://scai.fraunhofer.de/NDDUO#Epidemiological_study_type -->

#     <owl:Class rdf:about="&NDDUO;Epidemiological_study_type">
#         <rdfs:label>Epidemiological study</rdfs:label>
#         <rdfs:subClassOf rdf:resource="&span;Process"/>
#         <rdfs:subClassOf>
#             <owl:Restriction>
#                 <owl:onProperty rdf:resource="&NDDUO;is_entity_used_in"/>
#                 <owl:someValuesFrom rdf:resource="&NDDUO;epidemiology_context"/>
#             </owl:Restriction>
#         </rdfs:subClassOf>
#         <NDDUO:Synonym>Epidemiological studies</NDDUO:Synonym>
#     </owl:Class>

# 	<token id="NDDUO:Cluster" canonical="cluster">
# 		<variant base="aggregation"/>
# 		<variant base="clustered"/>
# 		<variant base="cluster"/>
# 	</token>

## type 3: bfo 
#     <!-- http://www.ifomis.org/bfo/1.1/span#TemporalInterval -->

#     <owl:Class rdf:about="&span;TemporalInterval">
#         <rdfs:label rdf:datatype="&xsd;string">temporal_interval</rdfs:label>
#         <rdfs:subClassOf rdf:resource="&span;ConnectedTemporalRegion"/>
#         <rdfs:comment rdf:datatype="&xsd;string">Definition: A connected temporal region [span:ConnectedTemporalRegion] lasting for more than a single moment of time.</rdfs:comment>
#         <rdfs:comment rdf:datatype="&xsd;string">Examples: any continuous temporal duration during which a process occurs</rdfs:comment>
#     </owl:Class>

# 	<token id="bfo:snap_DependentContinuant" canonical="dependent_continuant">
# 		<variant base="dependent_continuant"/>
# 	</token>


In [ ]:
# # errors met: 
# # when using beautifulsoup the <findall> method, the <owl:Class> tag can also be included in the <owl:Class> tag
# # exclude this situation by restraining the parents of <owl:Class> not being <owl:equivalentClass> or <rdfs:subClassOf>.
#     <!-- http://scai.fraunhofer.de/AlzheimerOntology#NMDA_receptor_antagonist_drug_molecule -->

#     <owl:Class rdf:about="&AlzheimerOntology;NMDA_receptor_antagonist_drug_molecule">
#         <rdfs:label>NMDA receptor antagonist drug molecule</rdfs:label>
#         <owl:equivalentClass>
#             <owl:Class>
#                 <owl:intersectionOf rdf:parseType="Collection">
#                     <rdf:Description rdf:about="&NDDUO;Molecular_entities"/>
# ...
#                 </owl:intersectionOf>
#             </owl:Class>
#         </owl:equivalentClass>
# ...
#     </owl:Class>

#     <owl:Class rdf:about="&AlzheimerOntology;CCR2">
#         <rdfs:label>CCR2</rdfs:label>
#         <rdfs:subClassOf rdf:resource="&AlzheimerOntology;Receptors"/>
#         <rdfs:subClassOf>
#             <owl:Class>
#                 <owl:intersectionOf rdf:parseType="Collection">
#                     <rdf:Description rdf:about="&AlzheimerOntology;chemokine"/>
# ...
#                 </owl:intersectionOf>
#             </owl:Class>
#         </rdfs:subClassOf>
#         <NDDUO:Synonym>CC chemokine receptor 2</NDDUO:Synonym>
#     </owl:Class>

In [1]:
ado_path = "/Users/yidesdo21/Projects/inputs/ontologies/Alzheimer Ontology v15R-xml_merged.owl"

In [2]:
from bs4 import BeautifulSoup
from xml.dom import minidom
import xmltodict
from collections import Counter, OrderedDict
from collections.abc import Mapping

In [3]:
with open(ado_path) as f:
    contents = f.read()
#     print(contents)

In [4]:
soup = BeautifulSoup(contents, 'html.parser')

In [12]:
# soup

In [14]:
# for owl_class in soup.find_all("owl:class"):
#     print(owl_class)

In [8]:
# create file
xmldoc = minidom.Document()

# creat root element
root_element = xmldoc.createElement('synonym')
xmldoc.appendChild(root_element)


for owl_class in soup.find_all("owl:class"):
    if not owl_class.findParent('owl:equivalentclass') and not owl_class.findParent('rdfs:subclassof'):
        rdf_about = owl_class["rdf:about"].replace("&", "")
        ontology = rdf_about.split(";")[0]
        token_id = rdf_about.replace(";", ":")
        
        productChild = xmldoc.createElement('token')
        productChild.setAttribute('id', token_id)  # attribute, value
        
        if child.name == "NDDUO:Synonym".lower():
            print(child.name)
            syno = child.get_text().lower()
#             break
            
#         for child in owl_class.contents:
#             variants = set()
#             if child.name == "rdfs:label": 
#                 cano = child.get_text()
#                 print(cano)
#                 break

In [18]:
# turn the first type AlzheimerOntology into the dictionary 
# I did two things here, these two methods to create the dictionary have the same outcome: 
# 1) create the xml dictioanry directly
# 2) I also abide by the same format as the nio, because then it is 
#    easier to do the comparison and the following merging into the dictionary 

# id_cano = list()   # [(token_id, canonical), (...),]
# id_syno = list()   # [(token_id1, synonym1), (token_id1, synonoym2), ...]

# create file
xmldoc = minidom.Document()

# creat root element
root_element = xmldoc.createElement('synonym')
xmldoc.appendChild(root_element)


for owl_class in soup.find_all("owl:class"):
    if not owl_class.findParent('owl:equivalentclass') and not owl_class.findParent('rdfs:subclassof'):
        rdf_about = owl_class["rdf:about"].replace("&", "")
        ontology = rdf_about.split(";")[0]
        token_id = rdf_about.replace(";", ":")
        
        productChild = xmldoc.createElement('token')
        productChild.setAttribute('id', token_id)  # attribute, value

        for child in owl_class.contents:
            variants = set()
            if child.name == "rdfs:label": 
                cano = child.get_text()
#                 cano = child.get_text().lower()

                if cano not in variants:   # avoid duplications in the variant base 
                    variants.add(cano)

                    productChild.setAttribute('canonical', cano)
                    root_element.appendChild(productChild)

                    # create child element, the canonical also has to be in the variant
                    product_grandChild = xmldoc.createElement('variant')

                    # insert user data into element
                    product_grandChild.setAttribute('base', cano)
                    productChild.appendChild(product_grandChild) 
                
#                 id_cano.append((token_id, cano))
                
            if child.name == "NDDUO:Synonym".lower():   # because beautifulsoup lowercase the children names
                syno = child.get_text()
#                 syno = child.get_text().lower()
#                 print(syno)
                
                if syno not in variants:  # avoid duplications in the variant base
                    variants.add(syno)
                    
                    # create child element
                    product_grandChild = xmldoc.createElement('variant')

                    # insert user data into element
                    product_grandChild.setAttribute('base', syno)
                    productChild.appendChild(product_grandChild)                  
                
#                 id_syno.append((token_id, syno))
        
xml_str = xmldoc.toprettyxml(indent ="\t")

# save file
save_path_file = "ado_case.xml"
dict_path = "/Users/yidesdo21/Projects/inputs/dictionary/"

with open(dict_path+save_path_file, "w") as f:
    f.write(xml_str)        


In [ ]:
# id_cano

In [86]:
# id_syno

In [5]:
## compare ado.xml to nio*.xml
xml_path = "/Users/yidesdo21/Projects/inputs/dictionary/"

with open(xml_path+"ado.xml") as f:
    ado_xml = f.read()
    
with open(xml_path+"nio_iri_v3_3.xml") as f:
    nio_xml = f.read()

ado_parsed = xmltodict.parse(ado_xml)
nio_parsed = xmltodict.parse(nio_xml)

In [6]:
ado_dict = ado_parsed["synonym"]["token"]
nio_dict = nio_parsed["synonym"]["token"]

In [7]:
ado_cnt, both_cnt = 0, 0
for ado in ado_dict:
    token_id = ado["@id"].lower()
#     print(token_id)
    ado_cnt += 1
    
    for nio in nio_dict:
        if nio["@id"].lower() == token_id:
#             print(nio)
#             print("----------")
            both_cnt += 1

print("Number of ADO concepts that are not in the NIO owl:", ado_cnt-both_cnt)
print("Number of ADO concepts that are in the NIO owl:", both_cnt)
print("Number of ADO concepts:", ado_cnt)

Number of ADO concepts that are not in the NIO owl: 1087
Number of ADO concepts that are in the NIO owl: 473
Number of ADO concepts: 1560


In [ ]:
## There are cases when the ADO concept and the NIO concept have the same token id
## but they have different canonicals or different variants. 
## No need to process the ADO dictionary, because it can be processed when merging the dictionaries together.

In [28]:
for ado in ado_dict:
    token_id = ado["@id"].lower()
    
    for nio in nio_dict:
        if nio["@id"].lower() == token_id:
            ## compare the case when token ids are the same in two dictionaries, but the canonicals are different
            if nio["@canonical"] != ado["@canonical"]:
                print("different in canonical")
                print(token_id)
                print("nio:", nio)
                print("ado:", ado)
                print("----------")
            
#             print(type(nio["variant"]), type(ado["variant"]))
#             print(isinstance(nio["variant"], Mapping))
#             print(nio["variant"])

            ## compare the case when token ids are the same in two dictionaries, but the variants are different
            ## when type(nio["variant"]) is not a list, i.e., is the OrderedDict, it means
            ## the concept only have one variant. The same goes for the ado variants.
            ## Then there are two cases: 
            ## 1) type(nio["variant"]) == OrderedDict and type(ado["variant"]) == OrderedDict ('collections.OrderedDict')
            ## nio concept has one variant, and ado concept has one variant, then compare them directly,
            ## 2) type(nio["variant"]) == list and type(ado["variant"]) == OrderedDict OR
            ##    type(nio["variant"]) == OrderedDict and type(ado["variant"]) == list
            ## one dictionary's concept and another has more than one concept, then no need to compare, they are different.
            
            
            ## for the concepts that have only one variant, the nio and ado dictionaries are the same.
            if isinstance(nio["variant"], Mapping) and isinstance(ado["variant"], Mapping):
#                 print(nio["variant"], ado["variant"])
                if nio["variant"] != ado["variant"]:
                    print("different in variants")
                    print(token_id)
                    print("nio:", nio["variant"])
                    print("ado:", ado["variant"])
            
            if type(nio["variant"]) != type(ado["variant"]):
                    print("different in variants")
                    print(token_id)
                    print("nio:", nio["variant"])
                    print("ado:", ado["variant"])
                    print("----------")
            
            
            if type(nio["variant"]) == list and type(ado["variant"]) == list:
#                 print(nio["variant"])
                nio_vars, ado_vars = list(), list()
                for v in nio["variant"]:
                    nio_vars.append(v['@base'])
                for v in ado["variant"]:
#                     print(ado["variant"])
                    ado_vars.append(v['@base'])
#                 print(sorted(nio_vars) == sorted(ado_vars))
                nio_variants, ado_variants = sorted(set(nio_vars)), sorted(set(ado_vars))
                if nio_variants != ado_variants:
                    print("different in variants")
                    print(token_id)
                    print("nio:", nio_variants)
                    print("ado:", ado_variants)
#                 print(nio["variant"])
#                 print(ado["variant"])
#                 print(OrderedDict(sorted(nio["variant"].iteritems(), key=lambda x: x[1]['@base'])))
#                 print(OrderedDict(sorted(ado["variant"].iteritems(), key=lambda x: x[1]['@base'])))
                    print("----------")
#             nio_var, ado_var = sorted(nio["variant"]), sorted(ado["variant"])
#             if nio_var != ado_var:
#                 print("different in variants")
#                 print("nio:", nio_var)
#                 print("ado:", ado_var)
#                 print("----------")
                
#             print(nio["variant"], ado["variant"])

#             print("----------")


different in variants
alzheimerontology:family_history
nio: ['family history', 'family history risk factors']
ado: ['family histories', 'family history', 'http://www.ebi.ac.uk/efo/efo_0000493']
----------
different in variants
alzheimerontology:frontotemporal_dementia
nio: [OrderedDict([('@base', 'dementia')]), OrderedDict([('@base', 'frontotemporal dementia')]), OrderedDict([('@base', 'frontotemporal lobar dementia')])]
ado: OrderedDict([('@base', 'frontotemporal dementia')])
----------
different in variants
alzheimerontology:cerebellar_cortex
nio: ['cerebellar cortex', 'cerebellar cortices', 'cerebelo']
ado: ['cerebellar cortex', 'cerebellar cortices']
----------
different in variants
alzheimerontology:frontotemporal_dementia
nio: OrderedDict([('@base', 'frontotemporal dementia')])
ado: [OrderedDict([('@base', 'frontotemporal dementia')]), OrderedDict([('@base', 'dementia')]), OrderedDict([('@base', 'frontotemporal lobar dementia')])]
----------
different in canonical
ndduo:prevalenc